In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pygam import LinearGAM, s
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.cross_decomposition import PLSRegression
from sklearn.feature_selection import RFE

def create_date_features(df):
    df['DayOfYear'] = df.index.dayofyear
    df['WeekOfYear'] = df.index.isocalendar().week
    df['Month'] = df.index.month
    df['Year'] = df.index.year

    # Sine-cosine encoding for cyclical features
    df['DayOfYear_sin'] = np.sin(2 * np.pi * df['DayOfYear'] / 365.25)
    df['DayOfYear_cos'] = np.cos(2 * np.pi * df['DayOfYear'] / 365.25)
    df['WeekOfYear_sin'] = np.sin(2 * np.pi * df['WeekOfYear'] / 52)
    df['WeekOfYear_cos'] = np.cos(2 * np.pi * df['WeekOfYear'] / 52)
    df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
    df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)

    return df

# Load the data
df = pd.read_csv('../../data/stock-market/ETFs/spy.us.txt')

# Convert Date column to datetime and set it as the index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Create lagged features
for i in range(12, 37):
    for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
        df[f'{col}_lag_{i}'] = df[col].shift(i)

# Drop rows with NaN values
df.dropna(inplace=True)

# Feature engineering: Extract date-related features and create sine-cosine embeddings
df = create_date_features(df)

# Define the target variable and features
y = df['Close']
features = [col for col in df.columns if col != 'Close']
X = df[features]

# Split the data into training, validation, and testing sets
end_date = df.index.max()
test_start_date = end_date - pd.DateOffset(years=1)
validation_start_date = test_start_date - pd.DateOffset(years=1)

train_X = X[X.index < validation_start_date]
train_y = y[y.index < validation_start_date]
validation_X = X[(X.index >= validation_start_date) & (X.index < test_start_date)]
validation_y = y[(y.index >= validation_start_date) & (y.index < test_start_date)]
test_X = X[X.index >= test_start_date]
test_y = y[y.index >= test_start_date]

# Normalize features
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)
train_X_normalized = X_normalized[X.index < validation_start_date]
validation_X_normalized = X_normalized[(X.index >= validation_start_date) & (X.index < test_start_date)]
test_X_normalized = X_normalized[X.index >= test_start_date]

# PLS dimensionality reduction
pls = PLSRegression(n_components=X.shape[1])  # Start with max components
pls.fit(train_X_normalized, train_y)
cumulative_variance = np.cumsum(np.sum(pls.x_scores_**2, axis=0) / np.sum(pls.x_scores_**2))
n_components = np.argmax(cumulative_variance >= 0.95) + 1  # Number of components for 95% variance

pls = PLSRegression(n_components=n_components)
X_pls = pls.fit_transform(X_normalized, y)[0]
train_X_pls = X_pls[:len(train_X)]
validation_X_pls = X_pls[len(train_X):len(train_X)+len(validation_X)]
test_X_pls = X_pls[len(train_X)+len(validation_X):]

# RFE feature selection
rfe = RFE(estimator=LinearRegression(), n_features_to_select=10)
X_rfe = rfe.fit_transform(X_pls, y)
train_X_rfe = X_rfe[:len(train_X)]
validation_X_rfe = X_rfe[len(train_X):len(train_X)+len(validation_X)]
test_X_rfe = X_rfe[len(train_X)+len(validation_X):]

# Save train, validation, and test sets after PLS and RFE transformations
pd.DataFrame(train_X_rfe).to_csv('train_X_pls_rfe.csv', index=False)
pd.DataFrame(validation_X_rfe).to_csv('validation_X_pls_rfe.csv', index=False)
pd.DataFrame(test_X_rfe).to_csv('test_X_pls_rfe.csv', index=False)
train_y.to_csv('train_y_pls_rfe.csv', index=False)
validation_y.to_csv('validation_y_pls_rfe.csv', index=False)
test_y.to_csv('test_y_pls_rfe.csv', index=False)

# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Additive Model': LinearGAM(s(0, n_splines=20, lam=0.1) + 
                s(1, n_splines=20, lam=0.1) + 
                s(2, n_splines=20, lam=0.1) + 
                s(3, n_splines=20, lam=0.1) + 
                s(4, n_splines=20, lam=0.1)),
    'SVR': SVR(kernel='linear', C=1, epsilon=0.1),
    'Random Forest': RandomForestRegressor(
        n_estimators=200,
        max_depth=10,
        max_features='sqrt',
        random_state=42
    ),
    'Neural Network': MLPRegressor(hidden_layer_sizes=(50, 50), max_iter=2000, learning_rate_init=0.001, alpha=0.01),
}

# Decomposition (Additive Model)
decomposition = seasonal_decompose(train_y, model='additive', period=365)
train_y_detrended = train_y - decomposition.trend.dropna()

# Train models and generate predictions using both training and validation data where applicable.
predictions = {}
for name, model in models.items():
    if name == 'Linear Regression':
        model.fit(train_X_rfe, train_y)
        pred_test = model.predict(test_X_rfe)
        pred_val = model.predict(validation_X_rfe)  # Predict on validation set for comparison later.
    else:
        model.fit(train_X_rfe if name != 'Additive Model' else train_X_rfe[:len(train_y_detrended)], 
                  train_y if name != 'Additive Model' else train_y_detrended)
        pred_test = model.predict(test_X_rfe)
        pred_val = model.predict(validation_X_rfe)  # Predict on validation set for comparison later.

    predictions[name] = {
        'Test': pred_test,
        'Validation': pred_val,
    }

# Add actual values for comparison in predictions dictionary.
for name in predictions.keys():
    predictions[name]['Actual_Test'] = test_y.values

# Calculate and display RMSE for each model on the test set.
for name in predictions.keys():
    rmse_test = np.sqrt(mean_squared_error(predictions[name]['Actual_Test'], predictions[name]['Test']))
    print(f"{name} Test RMSE: {rmse_test:.2f}")

# Plot actual vs. predicted prices for the last year.
plt.figure(figsize=(12, 8))
last_3_years = df.index[-1] - pd.DateOffset(years=3)
df_last_3_years = df[df.index >= last_3_years]
plt.plot(df_last_3_years.index, df_last_3_years['Close'], label='Close Price', color='blue')

# Plot each model's predictions on the test set.
for name in predictions.keys():
    plt.plot(test_y.index, predictions[name]['Test'], label=f"{name} Predictions")

plt.title("Actual vs Predicted Close Prices (Last Year with PLS and RFE)")
plt.xlabel("Date")
plt.ylabel("Close Price")
plt.legend()
plt.show()